In [1]:
import tensorflow as tf
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomBrightness
import os

In [4]:
# Set directory paths
train_dir = 'D:/LSCIDMR/Splitting_Augmented/train'
val_dir = 'D:/LSCIDMR/Splitting_Augmented/val'
test_dir = 'D:/LSCIDMR/Splitting_Augmented/test'

In [5]:
# Image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 16
num_classes = 8

In [6]:
# Enhanced data augmentation for the training set
data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
    RandomBrightness(0.1)
])

In [7]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int'
).map(lambda x, y: (data_augmentation(x, training=True), y))

Found 62979 files belonging to 8 classes.


In [8]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',
    shuffle=False
)

Found 7869 files belonging to 8 classes.
Found 7880 files belonging to 8 classes.


In [9]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int',
    shuffle=False  # Ensure the order is preserved
)


Found 7880 files belonging to 8 classes.


In [10]:
# Define the backbone of the model using EfficientNetB3
base_model = tf.keras.applications.EfficientNetB3(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = True 

In [11]:
# Add custom classification layers with Batch Normalization
from tensorflow.keras import regularizers
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu', 
                          kernel_regularizer=regularizers.l2(0.01)),  
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


initial_learning_rate = 0.0005 
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
import tensorflow as tf

class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, patience=7):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.best_weights = None
        self.best_loss = float('inf')
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        # Get training and validation accuracy and loss
        current_train_acc = logs.get("accuracy")
        current_val_acc = logs.get("val_accuracy")
        current_train_loss = logs.get("loss")
        current_val_loss = logs.get("val_loss")

        # Print training and validation loss and accuracy
        print(f"Epoch {epoch + 1}: "
              f"Training Loss = {current_train_loss:.4f}, Training Accuracy = {current_train_acc:.4f}, "
              f"Validation Loss = {current_val_loss:.4f}, Validation Accuracy = {current_val_acc:.4f}")

        # Early stopping check
        if current_val_loss < self.best_loss:
            self.best_loss = current_val_loss
            self.best_weights = self.model.get_weights()
            self.wait = 0  # reset wait counter if we have a new best loss
        else:
            self.wait += 1  # increment wait counter if no improvement
            if self.wait >= self.patience:
                print("Early stopping triggered!")
                self.model.stop_training = True
                # Restore the best weights if needed
                self.model.set_weights(self.best_weights)

# Instantiate the callback
custom_early_stopping = CustomEarlyStopping(patience=5)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[custom_early_stopping]
)


Epoch 1/30
3937/3937 [==============================] - 1344s 337ms/step - loss: 0.7508 - accuracy: 0.8674 - val_loss: 0.2850 - val_accuracy: 0.9285
Epoch 2/30
3937/3937 [==============================] - 1383s 351ms/step - loss: 0.3124 - accuracy: 0.9191 - val_loss: 0.2212 - val_accuracy: 0.9410
Epoch 3/30
3937/3937 [==============================] - 1329s 337ms/step - loss: 0.2233 - accuracy: 0.9428 - val_loss: 0.1457 - val_accuracy: 0.9626
Epoch 4/30
3937/3937 [==============================] - 1385s 352ms/step - loss: 0.1654 - accuracy: 0.9580 - val_loss: 0.1239 - val_accuracy: 0.9694
Epoch 5/30
3937/3937 [==============================] - 1324s 336ms/step - loss: 0.1238 - accuracy: 0.9680 - val_loss: 0.1049 - val_accuracy: 0.9732
Epoch 6/30
3937/3937 [==============================] - 1383s 351ms/step - loss: 0.0969 - accuracy: 0.9753 - val_loss: 0.0876 - val_accuracy: 0.9771
Epoch 7/30
3937/3937 [==============================] - 1333s 338ms/step - loss: 0.0799 - accuracy: 0.9804

In [17]:
model.save("C:/Users/Research Marshals/Satellite/models/8 classes/EfficiantNet_b3_latest_TRIAL_nw.h5")  # saves in HDF5 format


In [10]:
from tensorflow.keras.models import load_model

# Load the saved model
model_path = "C:/Users/Research Marshals/Satellite/models/8 classes/EfficiantNet_b3_latest_TRIAL.h5"
loaded_model = load_model(model_path)
print("Model loaded successfully!")


Model loaded successfully!


In [11]:
from sklearn.metrics import classification_report
import numpy as np

# Get true labels and predictions
y_true = np.concatenate([y.numpy() for _, y in test_ds], axis=0)
y_pred_probs = loaded_model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compute precision, recall, and F1-score
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=[f"Class {i}" for i in range(num_classes)], digits=4))


493/493 [==============================] - 26s 49ms/step
Classification Report:
              precision    recall  f1-score   support

     Class 0     0.9834    0.9801    0.9817       905
     Class 1     0.9720    0.9739    0.9730       998
     Class 2     0.9827    0.9688    0.9757      1057
     Class 3     0.9745    0.9663    0.9704       356
     Class 4     0.9902    0.9988    0.9945       809
     Class 5     0.9817    0.9836    0.9827      1527
     Class 6     0.9630    0.9849    0.9738       661
     Class 7     0.9942    0.9904    0.9923      1567

    accuracy                         0.9822      7880
   macro avg     0.9802    0.9809    0.9805      7880
weighted avg     0.9823    0.9822    0.9822      7880



In [28]:
for layer in model.layers:
    print(layer.name)


efficientnetb3
batch_normalization_1
global_average_pooling2d_1
dense_2
dropout_1
dense_3
